In [11]:
import requests

def get_user_token(address):
    url = f"https://scan.testnet.verichains.xyz/api?module=account&action=tokenbalance&address={address}&contractaddress=0x82a29828a3B5c16ed13Dc7840D0699746D2a741C"
    payload = ""
    headers = {}

    res = requests.request("GET", url, headers=headers, data=payload).json()

    return int(res["result"])

In [15]:
import requests
import json
import pandas as pd

def get_tshirt(id):
    url = "https://api.yoverse.io/marketplace/rpc"

    payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "vmp_listNFTBidding",
        "params": {
            "offset": 0,
            "limit": 10,
            "contractAddress": "0x5AF3a83b777cb1a862Efe672AFb645E32cEb7ff1",
            "tokenId": str(id)
        },
        "id": 1
    })
    headers = {}

    res = requests.request("POST", url, headers=headers, data=payload).json()

    bids = res["result"]["data"]
    if not bids:
        print(id, None)
        return
    bid = bids[0]
    price = int(bid["bidPrice"]) / 10**18
    address = bid["bidderAddress"]
    username = bid["bidderUserName"]
    star = get_user_token(address) / 10**18
    # df = pd.read_csv("starverse_users.csv")
    # lookup = dict(zip(df["address"], df["username"]))
    print(id, username, price, star)

get_tshirt(1)

In [19]:

import requests
import json
from collections import Counter

def get_min_price(c, k=1):
    url = "https://api.yoverse.io/marketplace/rpc"

    c_value = str(ord(c) - 65)

    payload = json.dumps({
        "jsonrpc": "2.0",
        "id": 1,
        "method": "vmp_searchNFT",
        "params": {
            "saleTypes": [
                "BUY_NOW"
            ],
            "contractAddress": [
                "0xd390b879e73EB4F64E037Bd9a054995D7553FC65"
            ],
            "limit": 10,
            "sort": {
                "sortType": "price",
                "sortField": "PRICE_LOW_TO_HIGH",
                "sortCurrency": "STAR"
            },
            "attrs": [
                {
                    "traitType": "Character",
                    "value": [
                        c_value
                    ]
                }
            ]
        }
    })
    headers = {}

    res = requests.request("POST", url, headers=headers, data=payload).json()

    stars = []

    for a in res["result"]["data"][:k]:
        stars.append(a["prices"]["STAR"])

    return stars


def get_text_price(text):
    print(text)
    s = 0
    for c, k in sorted(Counter(text).items()):
        cs = get_min_price(c, k)
        print(c, cs)
        s += sum(cs)
    print(s)


In [23]:
text = "EMBRACINGCHALLENGES"
# text = "ADVANCINGPARTNERSHIP"
# text = "UPHOLDINGINTEGRITY"
texts = [
    "EMBRACINGCHALLENGES",
    "ADVANCINGPARTNERSHIP",
    "UPHOLDINGINTEGRITY"
]

for text in texts:
    get_text_price(text)    

EMBRACINGCHALLENGES
A [189, 199]
B [99]
C [150, 150]
E [399, 400, 450]
G [250, 299]
H [400]
I [400]
L [100.1, 130]
M [100]
N [649, 700]
R [200]
S [60]
5324.1
ADVANCINGPARTNERSHIP
A [189, 199, 200]
C [150]
D [140]
E [399]
G [250]
H [400]
I [400, 490]
N [649, 700, 700]
P [199, 220]
R [200, 200]
S [60]
T [300]
V [99]
6144
UPHOLDINGINTEGRITY
D [140]
E [399]
G [250, 299]
H [400]
I [400, 490, 500]
L [100.1]
N [649, 700]
O [199]
P [199]
R [200]
T [300, 350]
U [99]
Y [200]
5874.1


In [24]:
ord("P") - 65

15

In [2]:
import requests
import json

def get_username(address):
    url = "https://api.yoverse.io/marketplace/rpc"

    payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "vmp_getProfile",
        "params": {
            "walletAddress": address
        },
        "id": 1
    })
    headers = {}
    res = requests.request("POST", url, headers=headers, data=payload).json()

    return res["result"]["data"]["username"]

get_username("0x10671C115BC5a74bc938B01E9c237f4a2EcD64b6")


'0x10671C'

In [32]:
import string
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [7]:
import requests

def get_token_holders(size=10):
    url = f"https://scan.testnet.verichains.xyz/api?module=token&action=getTokenHolders&contractaddress=0x82a29828a3B5c16ed13Dc7840D0699746D2a741C&offset={size}"

    payload = ""
    headers = {}

    res = requests.request("GET", url, headers=headers, data=payload).json()

    return res["result"]


In [15]:
import tqdm

def get_user(size=10):
    holders = get_token_holders(size)
    print(len(holders))
    for h in tqdm.tqdm(holders):
        h["star"] = int(h["value"])/10**18
        h["username"] = get_username(h["address"])
        h.pop("value")
    
    return holders

In [16]:
users = get_user(3000)

  0%|          | 0/1429 [00:00<?, ?it/s]

1429


100%|██████████| 1429/1429 [03:31<00:00,  6.76it/s]


In [29]:
import pandas as pd

def update_user():
    size = 3000

    df = pd.read_csv("starverse_users.csv")

    lookup = dict(zip(df["address"], df["username"]))
    
    holders = get_token_holders(size)
    print(len(holders))
    for h in tqdm.tqdm(holders):
        h["star"] = int(h["value"])/10**18
        h["username"] = lookup.get(h["address"])
        if not h["username"]:
            h["username"] = get_username(h["address"])
        h.pop("value")

    df = pd.DataFrame(holders)
    df.to_csv("starverse_users.csv", encoding="utf-8-sig", index=False)
    
    return holders

In [30]:
users = update_user()
len(users)

100%|██████████| 1429/1429 [00:00<00:00, 178840.50it/s]

1429


1429

In [20]:
import pandas as pd
df = pd.DataFrame(users)
df.to_csv("starverse_users.csv", encoding="utf-8-sig", index=False)

In [25]:
len([u for u in users if u["username"].lower() in u["address"].lower()])

354

In [24]:
len(users)

1429